In [1]:
import pandas as pd,numpy as np,tensorflow as tf
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import feather

In [4]:
def gen_csv_feather(path,path_new):
    f = open(path)
    reader = pd.read_csv(f, sep=',', iterator=True)
    loop = True
    chunkSize = 10000
    chunks = []
    while loop:
        try:
            chunk = reader.get_chunk(chunkSize)
            chunks.append(chunk)
        except StopIteration:
            loop = False
            print("Iteration is stopped.")
    df = pd.concat(chunks, ignore_index=True)
    print(df.count())
    feather.write_dataframe(df,path_new)

###测试集、训练集维数分别是（102277，3），（102277，4），需要先转化成feather文件，再进行批次读取

In [ ]:
gen_csv_feather(r"D:\Competition\Daguan\new_data\train_set.csv",r"D:\Competition\Daguan\new_data\train_set.feather")
gen_csv_feather(r"D:\Competition\Daguan\new_data\test_set.csv",r"D:\Competition\Daguan\new_data\test_set.feather")

In [3]:
train=feather.read_dataframe(r"D:\Competition\Daguan\new_data\train_set.feather")
test=feather.read_dataframe(r"D:\Competition\Daguan\new_data\test_set.feather")
#转化时列名出现乱码，修改即可
train=train.rename(columns={'锘縤d':'id'})
test=test.rename(columns={'锘縤d':'id'})

In [8]:
tfidf=TfidfVectorizer(ngram_range=(1,2),min_df=3, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
train_word=tfidf.fit_transform(train['word_seg']).toarray()
test_word=tfidf.transform(test['word_seg']).toarray()
from sklearn import preprocessing
ohe=preprocessing.OneHotEncoder(n_values=20)
l=train['class']
l=l.reshape(-1,1)
train_l=ohe.fit_transform(l).toarray()
train_l.shape

####训练集train_word,train_l  测试集test_word

In [28]:
def gen_batch(data,label,batch_size):
    row, col = data.shape
    n = row // batch_size     
    for i in range(n):
        x = data[i * batch_size : (i + 1) * batch_size,:]     
        y = label[i * batch_size : (i + 1) * batch_size,:] 
        yield (x, y)

In [1]:
learning_rate = 1e-3
num_units = 256
num_layer = 3
input_size = 199
time_step = 28
total_steps = 150
category_num = 20
steps_per_validate = 50
batch_size = tf.placeholder(tf.int32, [])
keep_prob = tf.placeholder(tf.float32, [])

In [52]:
import tensorflow as tf
x = tf.placeholder(tf.float32, [None, 5572])
y_label = tf.placeholder(tf.float32, [None, 20])
#每个样本按time_stepxinput_size输入LSTM网络
x_shape = tf.reshape(x, [-1, time_step, input_size])
#DroupoutWrapper防止过拟合
from tensorflow.contrib.rnn import DropoutWrapper
def cell(num_units):
    cell = tf.nn.rnn_cell.BasicLSTMCell(num_units=num_units)
    return DropoutWrapper(cell, output_keep_prob=keep_prob)

Instructions for updating:
Use the retry module or similar alternatives.


In [53]:
#三层LSTM
cells = tf.nn.rnn_cell.MultiRNNCell([cell(num_units) for _ in range(num_layer)])
#由于样本集比较大，设置batch_size=2000
h0 = cells.zero_state(batch_size, dtype=tf.float32)
output, hs = tf.nn.dynamic_rnn(cells, inputs=x_shape, initial_state=h0)
#网络中每个样本最后一个时间序列的结果作为输出
output = output[:, -1, :]
# Output Layer，全连接层线性变换
w = tf.Variable(tf.truncated_normal([num_units, category_num], stddev=0.1), dtype=tf.float32)
b = tf.Variable(tf.constant(0.1, shape=[category_num]), dtype=tf.float32)
y = tf.matmul(output, w) + b
# Loss
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.



In [ ]:
training= tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cross_entropy)

correction_prediction = tf.equal(tf.argmax(y, axis=1), tf.argmax(y_label, axis=1))
accuracy = tf.reduce_mean(tf.cast(correction_prediction, tf.float32))

saver=tf.train.Saver(max_to_keep=1)
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for step in range(total_steps + 1):
        for batch_x,batch_y in gen_batch(train_word,train_l,2000):
            sess.run(training, feed_dict={x: batch_x, y_label: batch_y, keep_prob: 0.5, batch_size:2000})
            if step % steps_per_validate == 0:
                print('Train', step, sess.run(accuracy, feed_dict={x: batch_x, y_label: batch_y, keep_prob:0.5,
                                                               batch_size:2000}))
            saver.save(sess,r"C:\Users\hp\tfModels\daguanLSTM.cpkt")

In [ ]:
testshape=test.shape[0]
with tf.Session() as sess2:
    sess2.run(tf.global_variables_initializer())
    saver.restore(sess2,r"C:\Users\hp\tfModels\daguanLSTM.cpkt")
    y_pred=sess2.run(y,feed_dict={x:test_word,keep_prob:0.5,batch_size:testshape})

In [ ]:
label_pred=tf.argmax(y_pred,axis=1)
with tf.Session() as result:
    l=result.run(label_pred)

In [ ]:
def get_predict(testdata,y_pred):
    user=pd.DataFrame(testdata['user_id'])
    label=pd.DataFrame(y_pred)
    u=pd.concat([user,label],axis=1).rename(columns={0:'label'})
    r=u[u['label']==1]
    id=r['user_id']
    return id

In [ ]:
id=get_predict(test,l)